<a href="https://colab.research.google.com/github/yusufOcakoglu/Deciding_the_MVP/blob/main/Data_Collection_and_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extracting from HTML


In [ ]:
import pandas as pd
import os
import re # regex for extracting year from filename

# Mevcut çalışma dizenindeki tüm HTML dosyalarını bul
html_dosyalari = [f for f in os.listdir('.') if f.endswith('.html') and f.startswith('sezon')]

if not html_dosyalari:
    print("❌ Hata: Çalışma ortamında 'sezon' ile başlayan HTML dosyası bulunamadı.")
else:
    print(f"✅ Bulunan HTML dosyaları: {html_dosyalari}")

    for dosya_adi in html_dosyalari:
        try:
            print(f"\n{dosya_adi} okunuyor...")
            # HTML dosyasından tabloyu okur (encoding ekleniyor)
            tablolar = pd.read_html(dosya_adi, encoding='utf-8')

            # İlk tablo bizim fikstür tablosudur
            df = tablolar[0]

            # Gereksiz ara başlıkları temizle (Wk, Day vs tekrar edenleri)
            df = df[df['Wk'] != 'Wk']

            # Boş satırları at (Bazen lig araları boş satır olur)
            df = df.dropna(subset=['Home'])

            print("✅ Tablo başarıyla okundu!")
            print(f"Toplam Maç Sayısı: {len(df)}")

            # Hakem (Referee) sütununu kontrol et
            if 'Referee' in df.columns:
                print("✅ Hakem sütunu mevcut!")
                # Sadece ihtiyacımız olan sütunları seçip kaydedelim
                df_clean = df[['Date', 'Home', 'Away', 'Score', 'Referee']]

                # Yıl bilgisini dosya adından çek ve CSV adını oluştur
                match = re.search(r'sezon(\d{2})_(\d{2})\.html', dosya_adi)
                if match:
                    year_part = match.group(1) + match.group(2)
                    csv_dosya_adi = f"clean_referee_{year_part}.csv"
                else:
                    csv_dosya_adi = f"clean_referee_{os.path.splitext(dosya_adi)[0]}.csv"

                # CSV olarak kaydederken encoding ekleniyor
                df_clean.to_csv(csv_dosya_adi, index=False, encoding='utf-8-sig') # Encoding changed back to utf-8
                print(f"💾 '{csv_dosya_adi}' olarak kaydedildi.")
                print(df_clean.head())
            else:
                print("⚠️ Tabloda 'Referee' sütunu bulunamadı. Bu dosya için CSV kaydedilemedi.")

        except Exception as e:
            print(f"❌ '{dosya_adi}' işlenirken bir hata oluştu: {e}")

✅ Bulunan HTML dosyaları: ['sezon20_21.html', 'sezon24_25.html', 'sezon21_22.html', 'sezon19_20.html', 'sezon17_18.html', 'sezon23_24.html', 'sezon18_19.html', 'sezon22_23.html']

sezon20_21.html okunuyor...
✅ Tablo başarıyla okundu!
Toplam Maç Sayısı: 420
✅ Hakem sütunu mevcut!
💾 'clean_referee_2021.csv' olarak kaydedildi.
         Date              Home           Away Score               Referee
0  2020-09-11          Rizespor     Fenerbahçe   1–2        Arda Kardeşler
1  2020-09-12  Fatih Karagümrük  Yeni Mal'spor   3–0    Abdulkadir Bitigen
2  2020-09-12         Sivasspor     Alanyaspor   0–2         Ali Palabıyık
3  2020-09-12           Göztepe    Denizlispor   5–1  Tugay Kaan Numanoğlu
4  2020-09-12       Galatasaray   Gaziantep FK   3–1       Bahattin Şimşek

sezon24_25.html okunuyor...
✅ Tablo başarıyla okundu!
Toplam Maç Sayısı: 342
✅ Hakem sütunu mevcut!
💾 'clean_referee_2425.csv' olarak kaydedildi.
         Date         Home             Away Score              Referee
0  202

# Combining Referee Data


In [ ]:
import pandas as pd
import glob

# 'clean_referee_' ile başlayan tüm CSV dosyalarını bul
referee_files = glob.glob("clean_referee_*.csv")

if not referee_files:
    print("❌ Hata: 'clean_referee_' ile başlayan hiçbir CSV dosyası bulunamadı.")
else:
    print(f"✅ Bulunan hakem CSV dosyaları: {referee_files}")
    all_referee_data = []

    for file in referee_files:
        try:
            df = pd.read_csv(file, encoding='utf-8')
            all_referee_data.append(df)
        except Exception as e:
            print(f"❌ '{file}' okunurken bir hata oluştu: {e}")

    if all_referee_data:
        # Tüm hakem verilerini birleştir
        combined_referee_df = pd.concat(all_referee_data, ignore_index=True)

        # Yeni CSV olarak kaydet
        output_file_name = "Clean_Referee_All_Seasons.csv"
        combined_referee_df.to_csv(output_file_name, index=False, encoding='utf-8-sig')
        print(f"\n✅ Tüm hakem verileri '{output_file_name}' olarak birleştirildi ve kaydedildi.")
        print(combined_referee_df.head())
    else:
        print("❌ Hiçbir hakem verisi birleştirilemedi.")

✅ Bulunan hakem CSV dosyaları: ['clean_referee_2324.csv', 'clean_referee_2223.csv', 'clean_referee_2425.csv', 'clean_referee_2122.csv', 'clean_referee_1718.csv', 'clean_referee_1920.csv', 'clean_referee_2021.csv', 'clean_referee_1819.csv']

✅ Tüm hakem verileri 'Clean_Referee_All_Seasons.csv' olarak birleştirildi ve kaydedildi.
         Date         Home          Away Score           Referee
0  2023-08-11  Trabzonspor   Antalyaspor   1–0      Zorbay Küçük
1  2023-08-12    Kasımpaşa    Ankaragücü   3–2   Bahattin Şimşek
2  2023-08-12    Konyaspor  İstanbulspor   1–1      Burak Pakkan
3  2023-08-12  Kayserispor   Galatasaray   0–0  Halil Umut Meler
4  2023-08-12   Pendikspor     Hatayspor   1–5      Çağdaş Altay


# Clean_Stats_All_Season

In [ ]:
import pandas as pd
import glob

# Klasördeki tüm T1 (stats) dosyalarını bul (örn: stats_1718.csv, T1(17-18).csv)
dosyalar = glob.glob("*.csv") # Sen dosya adlarını 'stats_1718.csv' yapmıştın, ona göre de ayarlayabilirsin

# Sadece İhtiyacımız Olan Sütunlar
gerekli_sutunlar = [
    'Date', 'HomeTeam', 'AwayTeam',  # Kimlik Bilgileri
    'FTHG', 'FTAG', 'FTR',           # Skorlar
    'HY', 'AY', 'HR', 'AR',          # KARTLAR (Ana Hedef)
    'HF', 'AF',                      # FAULLER (Tolerans Hesabı İçin)
    'HST', 'AST', 'HC', 'AC'         # BASKI (Şut ve Korner - Normalizasyon İçin)
]

data_list = []

print("🧹 Veri Temizliği Başlıyor...")

for dosya in dosyalar:
    # Sadece istatistik dosyalarını okuyalım (Hakem dosyaları veya final dosyası karışmasın)
    # Eğer dosya adında 'stats' veya 'T1' geçiyorsa ve 'Final' geçmiyorsa oku
    if ("stats" in dosya or "T1" in dosya) and "Final" not in dosya and "ref" not in dosya:
        try:
            print(f"okunuyor: {dosya}")
            df = pd.read_csv(dosya)

            # Sütun filtreleme: Sadece listedekiler varsa al
            mevcut_sutunlar = [col for col in gerekli_sutunlar if col in df.columns]
            df_clean = df[mevcut_sutunlar].copy()

            # Tarih Düzeltme (Çok önemli, yoksa merge patlar)
            # football-data genelde DD/MM/YYYY kullanır
            df_clean['Date'] = pd.to_datetime(df_clean['Date'], format='%d/%m/%Y', errors='coerce')

            data_list.append(df_clean)

        except Exception as e:
            print(f"⚠️ Hata ({dosya}): {e}")

if data_list:
    # Tüm sezonları alt alta birleştir
    all_stats = pd.concat(data_list, ignore_index=True)

    # Kaydet
    all_stats.to_csv("Clean_Stats_All_Seasons.csv", index=False)
    print("\n✅ Temizlenmiş İstatistik Verisi Hazır: Clean_Stats_All_Seasons.csv")
    print(all_stats.head())
else:
    print("❌ Hiçbir istatistik dosyası bulunamadı.")


🧹 Veri Temizliği Başlıyor...
okunuyor: T1(24-25).csv
okunuyor: T1(19-20).csv
okunuyor: T1(22-23).csv
okunuyor: T1(21-22).csv
okunuyor: T1(20-21).csv
okunuyor: T1(23-24).csv
okunuyor: T1(17-18).csv
okunuyor: T1(18-19).csv

✅ Temizlenmiş İstatistik Verisi Hazır: Clean_Stats_All_Seasons.csv
        Date     HomeTeam       AwayTeam  FTHG  FTAG FTR   HY   AY   HR   AR  \
0 2024-08-09  Galatasaray      Hatayspor     2     1   H  0.0  3.0  0.0  0.0   
1 2024-08-10    Kasimpasa      Konyaspor     2     3   A  5.0  3.0  0.0  0.0   
2 2024-08-10  Antalyaspor         Goztep     0     0   D  2.0  1.0  0.0  0.0   
3 2024-08-10   Fenerbahce  Ad. Demirspor     1     0   H  3.0  2.0  0.0  0.0   
4 2024-08-11    Sivasspor    Trabzonspor     0     0   D  1.0  0.0  0.0  0.0   

     HF    AF   HST  AST    HC   AC  
0  13.0  17.0  11.0  4.0  12.0  2.0  
1   6.0   7.0   3.0  3.0   2.0  7.0  
2   9.0  24.0   3.0  2.0   4.0  6.0  
3  17.0  13.0   5.0  1.0   9.0  4.0  
4  14.0  12.0   5.0  2.0   5.0  7.0  


# Merging The Datasets and Feature Engineering :


In [ ]:
import pandas as pd
import os

big_4 = ['Galatasaray', 'Fenerbahçe', 'Beşiktaş', 'Trabzonspor']

print("🚀 BÜYÜK BİRLEŞTİRME BAŞLIYOR...\n")

try:
    # 1. Combined Dosyaları Oku
    df_stats = pd.read_csv("Clean_Stats_All_Seasons.csv")
    df_ref = pd.read_csv("Clean_Referee_All_Seasons.csv")

    # 2. Tarih Formatlarını Eşitle
    df_stats['Date'] = pd.to_datetime(df_stats['Date'])
    df_ref['Date'] = pd.to_datetime(df_ref['Date'])

    # 3. Takım İsimlerini Temizle (Eşleşme garantisi için)
    clean_map = {
        'Galatasaray SK': 'Galatasaray', 'Fenerbahce': 'Fenerbahçe',
        'Besiktas': 'Beşiktaş', 'Besiktas JK': 'Beşiktaş',
        'Trabzonspor': 'Trabzonspor',
        'Buyuksehyr': 'Başakşehir', 'Medipol Basaksehir': 'Başakşehir', 'Basaksehir': 'Başakşehir',
        'Goztep': 'Göztepe', 'Ad. Demirspor': 'Adana Demirspor', # New additions from combined stats
        'Kasimpasa': 'Kasımpaşa' # New addition from combined stats
    }
    df_stats['HomeTeam'] = df_stats['HomeTeam'].replace(clean_map)
    df_stats['AwayTeam'] = df_stats['AwayTeam'].replace(clean_map)
    df_ref['Home'] = df_ref['Home'].replace(clean_map)
    df_ref['Away'] = df_ref['Away'].replace(clean_map)

    # 4. MERGE (Birleştirme)
    master_df = pd.merge(
        df_stats,
        df_ref,
        left_on=['Date', 'HomeTeam'],
        right_on=['Date', 'Home'],
        how='inner'
    )

    # Gereksiz tekrar sütununu sil
    if 'Home' in master_df.columns: master_df.drop(columns=['Home'], inplace=True)

    print(f"✅ Veriler başarıyla birleştirildi: {len(master_df)} maç")

    # 5. 'Season' Sütununu Türet
    # Futbol sezonları genellikle Ağustos'ta başlar, Mayıs'ta biter.
    # Örn: 2023-08-11 -> 2324, 2024-05-15 -> 2324, 2024-08-09 -> 2425
    year_start = master_df['Date'].dt.year
    year_end = master_df['Date'].dt.year + 1

    master_df.loc[master_df['Date'].dt.month < 8, 'Season'] = (year_start - 1).astype(str).str[2:] + year_start.astype(str).str[2:]
    master_df.loc[master_df['Date'].dt.month >= 8, 'Season'] = year_start.astype(str).str[2:] + year_end.astype(str).str[2:]
    master_df['Season'] = master_df['Season'].astype(str) # Ensure it's string type

    # 6. 'hasfans' Sütununu Ekle
    master_df['hasfans'] = 1 # Varsayılan olarak seyircili

    # 19-20 sezonunda 17-03-2020'den sonraki maçlar seyircisiz
    master_df.loc[
        (master_df['Season'] == '1920') & (master_df['Date'] > '2020-03-17'),
        'hasfans'
    ] = 0

    # 20-21 sezonunun tamamı seyircisiz
    master_df.loc[
        (master_df['Season'] == '2021'),
        'hasfans'
    ] = 0

except Exception as e:
    print(f"❌ Veri birleştirilirken bir hata oluştu: {e}")
    exit()

# --- FEATURE ENGINEERING (ANALİZ İÇİN SÜTUNLAR) ---

# 1. Big 4 İşaretlemesi
master_df['Home_is_Big4'] = master_df['HomeTeam'].apply(lambda x: 1 if x in big_4 else 0)
master_df['Away_is_Big4'] = master_df['AwayTeam'].apply(lambda x: 1 if x in big_4 else 0)

# 2. Tolerans Endeksi (Faul / Kart)
# Kart sayıları (Sarı + Kırmızı)
master_df['Home_Total_Cards'] = master_df['HY'] + master_df['HR']
master_df['Away_Total_Cards'] = master_df['AY'] + master_df['AR']

# Sıfıra bölünmeyi önlemek için, kart yoksa tolerans = faul sayısı
master_df['Home_Foul_Tolerance'] = master_df.apply(
    lambda x: x['HF'] / x['Home_Total_Cards'] if x['Home_Total_Cards'] > 0 else x['HF'], axis=1
)
master_df['Away_Foul_Tolerance'] = master_df.apply(
    lambda x: x['AF'] / x['Away_Total_Cards'] if x['Away_Total_Cards'] > 0 else x['AF'], axis=1
)

# 3. Baskı Endeksi (Şut + Korner)
# HST (İsabetli Şut) varsa onu kullan, yoksa HS (Toplam Şut)
if 'HST' in master_df.columns:
    master_df['Home_Pressure'] = master_df['HST'] + master_df['HC']
    master_df['Away_Pressure'] = master_df['AST'] + master_df['AC']
else:
    # This 'else' block might be removed if HST/AST are always present after cleaning
    # but kept for robustness if source files change
    master_df['Home_Pressure'] = master_df['HS'] + master_df['HC']
    master_df['Away_Pressure'] = master_df['AS'] + master_df['AC']

# --- KRONOLOJİK SIRALAMA ---
master_df = master_df.sort_values(by='Date').reset_index(drop=True)

# --- KAYDET ---
output_name = "TURKISH_SUPER_LIG_FULL_DATASET.csv"
master_df.to_csv(output_name, index=False, encoding='utf-8-sig')
print(f"\n🎉 İŞLEM TAMAM! Master dosya oluşturuldu: {output_name}")
print("Sütunlar:", master_df.columns.tolist())

🚀 BÜYÜK BİRLEŞTİRME BAŞLIYOR...

✅ Veriler başarıyla birleştirildi: 1962 maç

🎉 İŞLEM TAMAM! Master dosya oluşturuldu: TURKISH_SUPER_LIG_FULL_DATASET.csv
Sütunlar: ['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HY', 'AY', 'HR', 'AR', 'HF', 'AF', 'HST', 'AST', 'HC', 'AC', 'Away', 'Score', 'Referee', 'Season', 'hasfans', 'Home_is_Big4', 'Away_is_Big4', 'Home_Total_Cards', 'Away_Total_Cards', 'Home_Foul_Tolerance', 'Away_Foul_Tolerance', 'Home_Pressure', 'Away_Pressure']


In [ ]:
import pandas as pd

# Oluşturulan CSV dosyasını oku
full_data = pd.read_csv('TURKISH_SUPER_LIG_FULL_DATASET.csv')

# İlk 5 satırı göster
display(full_data.head())

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HY,AY,HR,AR,...,Season,hasfans,Home_is_Big4,Away_is_Big4,Home_Total_Cards,Away_Total_Cards,Home_Foul_Tolerance,Away_Foul_Tolerance,Home_Pressure,Away_Pressure
0,2018-08-11,Rizespor,Kasımpaşa,2,3,A,2.0,1.0,0.0,0.0,...,1819,1,0,0,2.0,1.0,6.500000,14.000000,8.0,12.0
1,2018-08-11,Fenerbahçe,Bursaspor,2,1,H,3.0,3.0,0.0,0.0,...,1819,1,1,0,3.0,3.0,4.000000,5.000000,6.0,7.0
2,2018-08-11,Sivasspor,Alanyaspor,1,0,H,1.0,3.0,0.0,0.0,...,1819,1,0,0,1.0,3.0,13.000000,5.333333,9.0,10.0
3,2018-08-12,Beşiktaş,Akhisar Belediyespor,2,1,H,3.0,6.0,0.0,0.0,...,1819,1,1,0,3.0,6.0,5.666667,2.666667,16.0,7.0
4,2018-08-12,Başakşehir,Trabzonspor,2,0,H,2.0,2.0,0.0,0.0,...,1819,1,0,1,2.0,2.0,8.000000,7.500000,14.0,2.0


# Cleaning and Handling the Missing Matches


In [ ]:
import pandas as pd

# 1. Mevcut Master Dosyayı Oku
df = pd.read_csv("TURKISH_SUPER_LIG_FULL_DATASET.csv")

print(f"Temizlik Öncesi Maç Sayısı: {len(df)}")

# 2. Eksik Verileri Sil (Dropna)
# Hakem veya Sarı Kart verisi olmayan maç, analiz edilemez maçtır.
# 'subset' parametresi ile sadece kritik sütunlara bakıyoruz.
df_clean = df.dropna(subset=['Referee', 'HY', 'AY', 'HF', 'AF'])

print(f"Temizlik Sonrası Maç Sayısı: {len(df_clean)}")
print(f"Silinen Maç Sayısı: {len(df) - len(df_clean)}")

# 3. Kontrol: Hatayspor'un 2023 Mart sonrasındaki maçları gitti mi?
hatay_check = df_clean[
    (df_clean['Season'] == '2223') &
    ((df_clean['HomeTeam'] == 'Hatayspor') | (df_clean['AwayTeam'] == 'Hatayspor')) &
    (df_clean['Date'] > '2023-02-06')
]

if len(hatay_check) == 0:
    print("✅ Deprem sonrası oynanmayan maçlar başarıyla temizlendi.")
else:
    print(f"⚠️ Dikkat: Hala {len(hatay_check)} adet şüpheli Hatayspor maçı var.")

# 4. Temiz Dosyayı Kaydet (Üzerine yazabiliriz veya yeni isim verebiliriz)
df_clean.to_csv("TURKISH_SUPER_LIG_FULL_DATASET.csv", index=False, encoding='utf-8-sig')
print("💾 Dosya güncellendi ve kaydedildi.")

Temizlik Öncesi Maç Sayısı: 1962
Temizlik Sonrası Maç Sayısı: 1941
Silinen Maç Sayısı: 21
✅ Deprem sonrası oynanmayan maçlar başarıyla temizlendi.
💾 Dosya güncellendi ve kaydedildi.
